In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import ast
import time
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer



##1. Συλλογή δεδομένων / scraping 

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

In [3]:
response = requests.get("https://www.news247.gr/loatki", headers=headers)
loatki_doc = BeautifulSoup(response.text)

KeyboardInterrupt: ignored

In [ ]:
headlines = loatki_doc.find_all('h2')

for headline in headlines:
    print(headline.text)

In [ ]:
links = loatki_doc.select("h2 a") 
links

In [ ]:
urls = [link['href'] for link in links]
urls

In [ ]:
leadtexts= loatki_doc.find_all('p' , {'class':'article__leadtext'})
summaries = [leadtext.text for leadtext in leadtexts]
#summaries

In [ ]:
dates = loatki_doc.select ('time')
time = [date['datetime'] for date in dates]


In [ ]:
titles = [link.text for link in links]
titles

In [ ]:
from bs4 import BeautifulSoup
import requests
import time

headers = {'user-agent': 'my-app/0.0.1'}
PAUSE_TIME = 3.5

def get_text(url):
    r = requests.get(url, headers = headers)
    time.sleep(PAUSE_TIME)
    s = BeautifulSoup(r.content, 'html.parser')
    link = s.find("h2 a")
    
    return([s, link])

In [ ]:
scraped_links = []
titles = []
counter = 0
base_url = "https://www.news247.gr/loatki?pages="
for i in range(1,20):
    url = base_url + str(i)
    #print(url)
    page = requests.get(url, headers = headers)
    s = BeautifulSoup(page.content, 'html.parser')
    #print(s)
    links = s.select("h2 a")
    links = set(links)
    
    url = [link['href'] for link in links]
    
    #title = [link.text for link in links]
    
    
    print(url)

    for l in url:
        link = l
        print(l)
        scraped_links.append(l)


    
    # Τυπώνει το νούμερο των links που είναι scraped
print(len(scraped_links)) 

loatki_links = (set(scraped_links))

In [ ]:
scraped_links

In [ ]:
len(scraped_links)

In [ ]:
import pandas as pd
df = pd.DataFrame(scraped_links, columns= ['url'])

In [ ]:
loatki = pd.DataFrame(loatki_links, columns= ['url'])

In [ ]:
loatki.to_csv("loatki.csv", index=False)
loatki = pd.read_csv('loatki.csv')
loatki

In [ ]:
urls = loatki['url'].tolist()
urls[0:100]

In [ ]:
len(urls)

In [ ]:
response = requests.get("https://www.news247.gr/loatki", headers=headers)

doc = BeautifulSoup(response.text)

In [ ]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
import numpy as np

In [ ]:
loatki = pd.DataFrame(columns = ['Title', 'created_at', 'URL', 'text', 'hashtags', 'About_URL', ])

In [ ]:
loatki

In [ ]:
titles = []#check
links = []#check
authors = [] # authors don't exist
dates = []#check
descs = []#check
hashtags = []#check
abouts_URL = []#check


for i in range(1,20):
  my_url = 'https://www.news247.gr/loatki?pages='+str(i)
  req = Request(my_url,headers={'User-Agent': 'Mozilla/5.0'})
  # print(my_url)
  raw_html = urlopen(req).read()
  news = BeautifulSoup(raw_html, "html.parser")


  titles_soup = news.find_all("h2", {"class": "article__title bold"})

  # links = news.find_all("h2", {"class": "article__title bold"})
  for title in titles_soup:
    links.append(title.find('a')['href'])
    titles.append((title.text).replace('\n',''))

  abouts_soup = news.find_all("div", {"class": "article__summary"})
  for about in abouts_soup:
    info = (about.text).splitlines()
    # print(info)
    hashtags.append(info[1])

    descs.append(info[2])

    dates.append(info[3])

    abouts_URL.append(about.find('a')['href'])

print(len(titles))
print(len(links))
print(len(dates))
print(len(descs))
print(len(hashtags))
print(len(abouts_URL))

In [ ]:
for i in range(0, len(titles)):
  loatki = loatki.append({'Title' : titles[i], 'created_at' : dates[i], 'URL' : links[i], 'text' : descs[i], 'hashtags' : hashtags[i], 'About_URL' : abouts_URL[i]}, ignore_index=True)

In [ ]:
loatki

In [ ]:
loatki.dtypes

In [ ]:
loatki_doc = loatki[['Title','created_at', 'URL', 'text', 'hashtags' ]]
loatki_doc

In [ ]:
loatki_doc['Year'] = loatki_doc.created_at.apply(lambda dt: dt.split(' ')[2])
loatki_doc['Month'] = loatki_doc.created_at.apply(lambda dt: dt.split(' ')[1])
loatki_doc['Day'] = loatki_doc.created_at.apply(lambda dt: dt.split(' ')[0])
loatki_doc['Time'] = loatki_doc.created_at.apply(lambda dt: dt.split(' ')[3])

months = {
    'Ιανουαρίου' : '01',
    'Φεβρουαρίου' : '02', 
    'Μαρτίου' : '03',
    'Απριλίου' : '04',
    'Μαΐου' : '05',
    'Ιουνίου': '06',
    'Ιουλίου' : '07',
    'Αυγούστου' : '08',
    'Σεπτεμβρίου' : '09',
    'Οκτωβρίου' : '10',
    'Νοεμβρίου' : '11',
    'Δεκεμβρίου' : '12',
     
}

loatki_doc['Date'] = loatki_doc.apply(lambda row: f'{months[row.Month]}/{row.Day}/{row.Year} {row.Time}', axis='columns',result_type='expand')
loatki_doc['Date'] = pd.to_datetime(loatki_doc['Date'])


loatki_doc

In [ ]:
loatki_doc.info() #den exw adeia kelia 

In [ ]:
loatki_doc.dtypes

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
loatki_doc.to_csv("/content/gdrive/My Drive/Colab Notebooks/loatki.tsv", sep='\t')

## 3. Καθάρισμα/Γραφήματα

In [ ]:
loatki_doc.dtypes

In [ ]:
tags = loatki_doc.groupby(['hashtags']).count()['Title'].sort_values(ascending=False)


Day_data = loatki_doc.groupby(['Day']).count()['Title']
(Day_data,tags)

In [ ]:
loatki_doc['text'] = loatki_doc['text'].str.lower()
loatki_doc['text'][5]

In [ ]:
loatki_doc.dtypes

In [ ]:
#loatki_doc['Year'] = loatki_doc['Year'].astype(np.int64)
#loatki_doc['Month'] = loatki_doc['Month'].astype(np.int64)
#loatki_doc['Day'] = loatki_doc['Day'].astype(np.int64)

In [ ]:
loatki_doc['text'].tail(5)

In [ ]:
loatki_doc['text'] = loatki_doc['text'].str.replace(r'[\n]*\s*https?:\/\/.*[\r\n]*'," ").str.replace(r'\n*',"").str.replace(r'[▪️🟢⚪👉]*',"").str.replace(r"\w\/\w", "")
loatki_doc['text'] = loatki_doc['text'].str.replace('amp;', '').str.replace(r'[\!\-\:\,\.«»\"“”\[–()&]'," ").str.replace('&gt;', ' ').str.replace(r"\s\s\s?", ' ')
loatki_doc['text'] = loatki_doc['text'].str.replace(r"\w*…", "")



In [ ]:
loatki_doc['text'].tail(5)

In [ ]:
plt.figure(figsize=(30,10))
plt.bar(tags.keys(),tags)
plt.show()

plt.figure(figsize=(30,10))
plt.bar(Day_data.keys(),Day_data)
plt.show()

In [ ]:
loatki_doc['hashtags'].value_counts().head(10)

In [ ]:
%matplotlib inline
loatki_doc['hashtags'].value_counts().head(10).sort_values().plot(kind='barh')

In [ ]:
pd.to_datetime(loatki_doc['Date']).dt.dayofweek 

In [ ]:
loatki_doc.set_index('Date' , inplace=True)


In [ ]:
loatki_doc.index

In [ ]:
loatki_doc['dayofweek']=pd.to_datetime(loatki_doc.index).dayofweek

In [ ]:
day_of_week = loatki_doc.groupby(['dayofweek']).count()['Title'].sort_values(ascending=False)


In [ ]:
day_of_week #ποιες μερες της εβδομάδας κάνουν τα περοσσότερα ποστ

In [ ]:
day_of_week.plot(kind='bar')

In [ ]:
loatki_doc.dtypes

In [ ]:
from collections import Counter
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
for token in doc:
    print(token, token.lemma_)

In [ ]:
' '.join(token.lemma_ for token in doc)

## wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
import spacy

In [ ]:
!python3 -m spacy download el_core_news_sm

In [ ]:
nlp = spacy.load("el_core_news_sm")

In [ ]:
doc = nlp(loatki['text'][1])

In [ ]:
list(nlp.Defaults.stop_words)[0:20]

In [ ]:
text = loatki_doc.text
stop = stopwords.words('greek')
stop = stop + ['το', 'ή', 'και','αλλά', 'τη', 'την', 'ο', 'ένα', 'να', 'της', 'όταν', 'με', 'γι', 'τους', 'object', 'με', 'για', 'αλλά', 'αν', 'αλλά', 'δε', 'δεν', 'των', 'ότι','πως']

# Word cloud image:
wordcloud = WordCloud(stopwords=stop, width = 2000, height = 1000, background_color = 'white').generate(str(text))

fig = plt.figure( figsize = (40, 30), facecolor = 'k', edgecolor = 'k')

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:

word_count = Counter(" ".join(loatki_doc["text"]).split())
word_count = {word.lower() : count for word, count in word_count.items() if not word.lower() in stop}
word_count = pd.DataFrame(data={
    'word': word_count.keys(),
    'count': word_count.values()
})

word_count.sort_values('count',ascending=False).head(15)

In [ ]:
#diades leksewn
cv2 = CountVectorizer( stop_words= nlp.Defaults.stop_words, max_features=20, ngram_range=(2, 2) ) #token_pattern = '\\bκυ[α-ωά-ώ]*\\b') #, tokenizer=my_tokenizer)
count_vector2 = cv2.fit_transform(loatki_doc['text'])
results2 = pd.DataFrame(count_vector2.toarray(), columns=cv2.get_feature_names()) 
results2.head()

## 15 συχνότερες λέξεις

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [ ]:
count_vector = cv.fit_transform(loatki_doc['text'])

In [ ]:
count_vector.shape

In [ ]:
results = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names()) 

results.head()

In [ ]:
cv.vocabulary

In [ ]:
cv = CountVectorizer(stop_words= nlp.Defaults.stop_words, min_df=0.01, max_df=0.95)

In [ ]:
count_vector = cv.fit_transform(loatki_doc['text'])

In [ ]:
results = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names()) 
results.head()

In [ ]:
from html import unescape
def my_tokenizer(doc):
            
    # apply the preprocessing and tokenzation steps
    doc_clean = unescape(doc).lower()
    tokens = nlp(doc_clean)
    lemmatized_tokens = [token.lemma_ for token in tokens if (len(str(token.lemma_))>2)]
            
    # use CountVectorizer's _word_ngrams built in method
    # to remove stop words and extract n-grams
    return(lemmatized_tokens)

In [ ]:
cv = CountVectorizer(stop_words= nlp.Defaults.stop_words, min_df=0.01, max_df=0.95, tokenizer=my_tokenizer)
count_vector = cv.fit_transform(loatki_doc['text'])
results = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names()) 
results.head()

In [ ]:
cv = CountVectorizer( stop_words= nlp.Defaults.stop_words, max_features=20) #, tokenizer=my_tokenizer)
count_vector = cv.fit_transform(loatki_doc['text'])
results = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names()) 
results.head()

In [ ]:
sum_words =  results.sum(axis =0)
sum_words.sort_values(ascending = False).head(15) 

In [ ]:
cv2 = CountVectorizer( stop_words= nlp.Defaults.stop_words, max_features=20, ngram_range=(2, 2) ) #token_pattern = '\\bκυ[α-ωά-ώ]*\\b') #, tokenizer=my_tokenizer)
count_vector2 = cv2.fit_transform(loatki_doc['text'])
results2 = pd.DataFrame(count_vector2.toarray(), columns=cv2.get_feature_names()) 
results2.head()

In [ ]:
#oisixnoteres diades leksewn
sum_words2 =  results2.sum(axis =0)
sum_words2.sort_values(ascending = False).head(15) 

In [ ]:
w = sum_words.sort_values(ascending = False).head(15).plot(kind='bar')
w.spines['right'].set_visible(False)
w.spines['top'].set_visible(False)
w.set_title('15 πιο συχνές λέξεις στο θέμα λοατκι')

In [ ]:
w2 = sum_words2.sort_values(ascending = True).tail(15).plot(kind='barh', color='red')
w2.spines['right'].set_visible(False)
w2.spines['top'].set_visible(False)
w2.set_title('15 πιο συχνές δυάδες λέξεων στο θέμα ΛΟΑΤΚΙ')

In [ ]:
loatki_doc.groupby('Month')['Title'].count().plot(kind='bar',figsize=(10,7), color='tab:red',title="άρθρα ανά μήνα")

In [ ]:
loatki_doc.resample(rule='D').count()

In [ ]:
loatki_doc['Title'].groupby(loatki_doc['Year']).count()

## sentiment analysis

In [ ]:
import glob
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
filepath = "https://raw.githubusercontent.com/datajour-gr/Data_journalism/master/week10/NRC_GREEK_Translated_6_2020.csv"
emolex_df = pd.read_csv(filepath)
emolex_df = emolex_df.drop_duplicates(subset=['word'])
emolex_df = emolex_df.dropna()
emolex_df.reset_index(inplace = True)
emolex_df.tail(15)

In [ ]:
vec = CountVectorizer(analyzer = 'word', vocabulary = emolex_df.word, 
                      strip_accents = 'unicode',  
                      stop_words= nlp.Defaults.stop_words,
                      ngram_range=(1, 2))

In [ ]:
#για να ελεγξω μόνο τις λέξεις που υπάρχουν στο συναισθηματικό λεξικό 
matrix = vec.fit_transform(loatki_doc['text'])
vocab = vec.get_feature_names()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)
wordcount_df.head()

In [ ]:
len(wordcount_df)

In [ ]:
# anger words
angry_words = emolex_df[emolex_df.Anger == 1]['word']


# positive words
positive_words = emolex_df[emolex_df.Positive == 1]['word']


# sadness words
sadness_words = emolex_df[emolex_df.Sadness == 1]['word']


# surprise words
surprise_words = emolex_df[emolex_df.Surprise == 1]['word']


# disgust words
disgust_words = emolex_df[emolex_df.Disgust == 1]['word']


# anticipation words
anticipation_words = emolex_df[emolex_df.Anticipation == 1]['word']


# negative words
negative_words = emolex_df[emolex_df.Negative == 1]['word']


# joy words
joy_words = emolex_df[emolex_df.Joy == 1]['word']


# trust words
trust_words = emolex_df[emolex_df.Trust == 1]['word']


# fear words
fear_words = emolex_df[emolex_df.Fear == 1]['word']

In [ ]:
wordcount_df.shape

In [ ]:
df = df.replace(np.nan, '0')

In [ ]:
emolex_df.head()

In [ ]:
emolex_df[emolex_df.Anger == 1].tail()

In [ ]:
angry_words = emolex_df[emolex_df.Anger == 1]['word']
positive_words = emolex_df[emolex_df.Positive == 1]['word']
sadness_words = emolex_df[emolex_df.Sadness == 1]['word']
surprise_words = emolex_df[emolex_df.Surprise == 1]['word']
disgust_words = emolex_df[emolex_df.Disgust == 1]['word']
anticipation_words = emolex_df[emolex_df.Anticipation == 1]['word']
negative_words = emolex_df[emolex_df.Negative == 1]['word']
joy_words = emolex_df[emolex_df.Joy == 1]['word']
trust_words = emolex_df[emolex_df.Trust == 1]['word']
fear_words = emolex_df[emolex_df.Fear == 1]['word']

In [ ]:
wordcount_df

In [ ]:
loatki_doc['anger'] = np.array(wordcount_df[angry_words].sum(axis=1))
loatki_doc['positivity'] = np.array(wordcount_df[positive_words].sum(axis=1))
loatki_doc['joy'] = np.array(wordcount_df[joy_words].sum(axis=1))
loatki_doc['disgust'] = np.array(wordcount_df[disgust_words].sum(axis=1))
loatki_doc['surprise'] = np.array(wordcount_df[surprise_words].sum(axis=1))
loatki_doc['trust'] = np.array(wordcount_df[trust_words].sum(axis=1))
loatki_doc['anticipation'] = np.array(wordcount_df[anticipation_words].sum(axis=1))
loatki_doc['sadness'] = np.array(wordcount_df[sadness_words].sum(axis=1))
loatki_doc['negative'] = np.array(wordcount_df[negative_words].sum(axis=1))
loatki_doc['fear'] = np.array(wordcount_df[fear_words].sum(axis=1))
loatki_doc

In [ ]:
wordcount_df[joy_words].sum(axis=1)

In [ ]:
loatki_doc.dtypes

In [ ]:
loatki_doc['positivity'].sum() #Βαθμος θετικότητας στα συλλεγμένα tweets

In [ ]:
loatki_doc['negative'].sum() #βαθμός αρνητικότητας στα συλλεγμένα tweets

In [ ]:
emotions = loatki_doc.resample(rule='D').sum()
emotions.head()

In [ ]:
loatki_doc.set_index('created_at' , inplace=True) #Βάζουμε ως index το date

In [ ]:
emotions[['anger','sadness','negative','fear']].sum().plot() #αρχικά για τα αρνητικά

In [ ]:
emotions[['positivity','joy','trust','surprise','anticipation']].sum().plot() #αρχικά για τα αρνητικά

In [ ]:
loatki_doc['text'].str.len().groupby(by=loatki_doc.index).mean().plot() 

In [ ]:
t = loatki_doc[['anger','positivity','joy','surprise','trust','anticipation','sadness','negative','fear']].sum().sort_values(ascending= False).plot(kind='bar',color=('C0','C1','C2','C3','C4','C5','C6','C7','C9'), figsize=(10,5))
t.set_title('Συναισθήματα στον τρόπο γραφής των άρθρων  με θέμα ΛΟΑΤΚΙ')
t.spines['right'].set_visible(False)
t.spines['top'].set_visible(False)
t.set_xlabel('Emotions')

In [ ]:
ax = emotions['positivity'].resample('M').mean().plot(figsize=(16,4), 
                                                          color = 'g', label='pos')
emotions['negative'].resample('M').mean().plot(figsize=(16,4), ax = ax, 
                                                   label = 'neg', color = 'r')
emotions['trust'].resample('M').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'trust',color = 'blue').legend()
emotions['fear'].resample('M').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'fear',color = 'black').legend()
emotions['anger'].resample('M').mean().plot(figsize=(16,4), ax = ax, 
                                                label = 'anger',color = 'purple').legend()

In [ ]:
(rows,_) = emotions.shape
labels = emotions.columns

for i in range(rows):
    data = emotions.iloc[[i]].values[0].tolist()
    date = str(emotions.iloc[[i]].axes[0].values[0]).split('T')[0]
    plt.figure(figsize=(30,10))
    plt.pie(data,labels=labels,)
    plt.legend()
    plt.title(f' Τα συναισθήματα στα άρθρα για {date}')
    plt.show()

In [ ]:
from textblob import TextBlob, Word

#def detect_polarity(text):
#    return TextBlob(text).sentiment.polarity
def detect_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [ ]:
df.to_csv('news_clean.csv', index = False)

## Πολικότητα άρθρων ανά διαστήματα 

In [ ]:
emotions['2022':]


In [ ]:
emotions['2022-01': '2022-06']

In [ ]:
emotions['2022-01': '2022-06'].plot(y='positivity') #diagramma gia to sinaisthima tis xaras to prwto eksamino tou 2022


In [ ]:
emotions['2022-01': '2022-06'].plot(y='anger')

In [ ]:
emotions.resample('A').mean().plot(y='positivity')
#resample, group ta dedomena ana xronia

In [ ]:
emotions.resample('A').mean().plot(y='anger')
#resample, group ta dedomena ana xronia

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [ ]:
emotions['anger'].plot()
emotions['anger'].resample('A').mean().plot()
emotions['anger'].resample('5A').mean().plot()

In [ ]:
loatki_doc.sort_values(by='anger').head() #oi xeiroteroi mines


In [ ]:

emotions['positivity'].resample('A').max().plot(color='red')
emotions['positivity'].resample('A').min().plot(color='green')
emotions['positivity'].resample('A').mean().plot(color='blue')

In [ ]:

emotions['anger'].resample('A').max().plot(color='red')
emotions['anger'].resample('A').min().plot(color='green')
emotions['anger'].resample('A').mean().plot(color='blue')

## Heatmap

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words= nlp.Defaults.stop_words, max_features=50)
tfidf_matrix_train1 = tfidf_vectorizer.fit_transform(loatki_doc['Title'])

In [ ]:
resultsWb = pd.DataFrame(tfidf_matrix_train1.toarray(), columns=tfidf_vectorizer.get_feature_names()) 
resultsWb.head()

In [ ]:
resultsWb.sum(axis =0).sort_values(ascending = False)

In [ ]:
resultsTw = pd.DataFrame(tfidf_matrix_train2.toarray(), columns=tfidf_vectorizer.get_feature_names()) 
resultsTw.head()